In [5]:
import dask.dataframe as dd
from dask.multiprocessing import get
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [6]:
srna_file = 'data/21dpi_tgac/ShortStack_D.gff3'
df = pd.read_csv(srna_file,sep="\t",header=None)
df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
print('Total: ',len(df.index))

de_file = 'data/files/diffexpr-results.0.05.csv'
df_de_clus = pd.read_csv(de_file,sep=",")
print('Total DE: ',len(df_de_clus.index))


gene_list = df_de_clus.Gene.tolist()
def filter(s):
    s = s.split(';')[0].split('=')[1]
    for i in gene_list:
        if s == i:
            return True
    return False

#shortstack gff results that are DE
df_de = df[df.attribute.apply(lambda s: filter(s))]
print('Total DE filtered: ',len(df_de.index))

Total:  28845
Total DE:  43
Total DE filtered:  43


In [7]:
df_de.to_csv('data/files/mirna_de.csv',sep='\t',index=None)

In [10]:
#comes from X
froms = []

df_from = pd.read_csv('data/cross/repbase.gff',sep="\t")
df_from.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
froms.append(('repbase',df_from))

df_from = pd.read_csv('data/cross/trep.gff',sep="\t")
df_from.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
froms.append(('trep',df_from))

df_from = pd.read_csv('data/cross/tracker.gff',sep="\t")
df_from.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
froms.append(('tracker',df_from))

df_from = pd.read_csv('data/cross/genes.gff',sep="\t")
df_from.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
froms.append(('genes',df_from))

df_from = pd.read_csv('data/cross/promoters.gff',sep="\t")
df_from.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
froms.append(('promoters',df_from))

srna_type = []
srna_type.append(('all_srna',df))
srna_type.append(('de_srna',df_de))

distance = 10
for s_type in srna_type:
    s_name = s_type[0]
    s_df = s_type[1]
    for f_from in froms:
        df_from = f_from[1]
        from_name = f_from[0]
        filename = 'data/files/srna_' + s_name + '_' + from_name + '.csv'
        f = open(filename, 'w')
        f.close()
        def filter(x):
            start = x.start - distance
            end = x.end + distance
            df_res = df_from[(df_from.seqname == x.seqname) & (df_from.end >= start) & (df_from.start <= end)]
            if len(df_res.index) == 0:
                return False
            with open(filename, 'a') as f:
                f.write('\t'.join(x.map(str))) # sRNA header
                f.write('\n')
                f.write(df_res.to_string(header=False))
                f.write('\n')
                f.write('-' * 5)
                f.write('\n')
                f.close()
            return True

        ddata = dd.from_pandas(s_df, npartitions=7)
        res = ddata.map_partitions(lambda s_df: s_df.apply(lambda x: filter(x), axis=1)).compute(get=get)  
        df_res = s_df[res]
        print(s_name, from_name, len(df_res.index))

all_srna repbase 87
all_srna trep 155
all_srna tracker 2690
all_srna genes 4853
all_srna promoters 9394
de_srna repbase 0
de_srna trep 0
de_srna tracker 1
de_srna genes 12
de_srna promoters 7


In [145]:
#repbase  87
#Trep  155
#Tracker  2690
#Genes  5315

#de_srna repbase 0
#de_srna trep 0
#de_srna tracker 1
#de_srna genes 12

#DF all 1188
#de_srna repbase 0
#de_srna trep 0
#de_srna tracker 3
#de_srna genes 15
#de_srna promoters 11

In [ ]:
sys.path